In [ ]:
import glob
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import re
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.applications.vgg16 import VGG16
#from cv2 import imshow

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers.legacy import Adam

2023-06-07 22:45:46.490194: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 22:45:46.931578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-07 22:45:46.931634: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-07 22:45:48.387272: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Data pre-processing

In [ ]:
root_dir = '/home/haohao/Desktop/Hieu/VisualPhish/'
dataset_dir_benign = root_dir + 'benign_test'
dataset_dir_phishing = root_dir + 'phishing'
dataset_dir_trusted = root_dir + 'trusted_list'
benign = list(glob.glob(dataset_dir_benign + '/*'))
phishing = list(glob.glob(dataset_dir_phishing + '/**/*'))
trusted_list = list(glob.glob(dataset_dir_trusted + '/**/*'))

cnt = 0
for f in benign:
  if f.upper().find(".PNG") == -1 and f.upper().find(".JPG") == -1:
    try:
      os.remove(f)
      cnt += 1
    except OSError as e:
      print(e)

print("benign removed: " + str(cnt))
cnt = 0

for f in phishing:
  if f.upper().find(".PNG") == -1 and f.upper().find(".JPG") == -1:
    try:
      os.remove(f)
      cnt += 1
    except OSError as e:
      print(e)

print("phishing removed: " + str(cnt))
cnt = 0

for f in trusted_list:
  if f.upper().find(".PNG") == -1 and f.upper().find(".JPG") == -1:
    try:
      os.remove(f)
      cnt += 1
    except OSError as e:
      print(e)

print("trusted_list removed: " + str(cnt))

print('Number of benign images: ' + str(len(benign)))
print(benign[:5])
print('Number of phishing images: ' + str(len(phishing)))
print(phishing[:5])
print('Number of trusted images: ' + str(len(trusted_list)))
print(trusted_list[:5])

benign removed: 0
phishing removed: 0
trusted_list removed: 0
Number of benign images: 3107
['/home/haohao/Desktop/Hieu/VisualPhish/benign_test/02673.png', '/home/haohao/Desktop/Hieu/VisualPhish/benign_test/02357.png', '/home/haohao/Desktop/Hieu/VisualPhish/benign_test/00993.PNG', '/home/haohao/Desktop/Hieu/VisualPhish/benign_test/00340.png', '/home/haohao/Desktop/Hieu/VisualPhish/benign_test/01948.png']
Number of phishing images: 1195
['/home/haohao/Desktop/Hieu/VisualPhish/phishing/docmagic/T52_0.png', '/home/haohao/Desktop/Hieu/VisualPhish/phishing/allegro/T6_8.png', '/home/haohao/Desktop/Hieu/VisualPhish/phishing/allegro/T6_1.png', '/home/haohao/Desktop/Hieu/VisualPhish/phishing/allegro/T6_6.png', '/home/haohao/Desktop/Hieu/VisualPhish/phishing/allegro/T6_3.png']
Number of trusted images: 1870
['/home/haohao/Desktop/Hieu/VisualPhish/trusted_list/twitter/T136_71.png', '/home/haohao/Desktop/Hieu/VisualPhish/trusted_list/twitter/T136_85.png', '/home/haohao/Desktop/Hieu/VisualPhish/tru

# Load dataset and normalization

In [ ]:
img_height = 256
img_width = 256
phishing_size = len(phishing)
trusted_size = len(trusted_list)

In [ ]:
# We can use trusted_list with phishing labels

dataseta_train = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_phishing, validation_split=0.3, subset='training', seed=123, image_size=(img_height, img_width), batch_size=None, labels=[1 for i in range(phishing_size)], label_mode='int')
datasetb_train = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_benign, validation_split=0.3, subset='training', seed=123, image_size=(img_height, img_width), batch_size=None, labels=None)
datasetc_train = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_trusted, validation_split=0.3, subset='training', seed=123, image_size=(img_height, img_width), batch_size=None, labels=[1 for i in range(trusted_size)], label_mode='int')

dataseta_train = dataseta_train.map(lambda x, y: (x, tf.cast(y, tf.float32)))
datasetb_train = datasetb_train.map(lambda x: (x, tf.constant(0.0)))
datasetc_train = datasetc_train.map(lambda x, y: (x, tf.cast(y, tf.float32)))

dataseta_test = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_phishing, validation_split=0.3, subset='validation', seed=123, image_size=(img_height, img_width), batch_size=None, labels=[1 for i in range(phishing_size)], label_mode='int')
datasetb_test = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_benign, validation_split=0.3, subset='validation', seed=123, image_size=(img_height, img_width), batch_size=None, labels=None)
datasetc_test = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_trusted, validation_split=0.3, subset='validation', seed=123, image_size=(img_height, img_width), batch_size=None, labels=[1 for i in range(trusted_size)], label_mode='int')

dataseta_test = dataseta_test.map(lambda x, y: (x, tf.cast(y, tf.float32)))
datasetb_test = datasetb_test.map(lambda x: (x, tf.constant(0.0)))
datasetc_test = datasetc_test.map(lambda x, y: (x, tf.cast(y, tf.float32)))

Found 1195 files belonging to 155 classes.
Using 837 files for training.


2023-06-07 22:46:05.188502: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-07 22:46:05.188551: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-07 22:46:05.188596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (haohao-virtual-machine): /proc/driver/nvidia/version does not exist
2023-06-07 22:46:05.190789: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 3107 files belonging to 1 classes.
Using 2175 files for training.
Found 1870 files belonging to 155 classes.
Using 1309 files for training.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Found 1195 files belonging to 155 classes.
Using 358 files for validation.
Found 3107 files belonging to 1 classes.
Using 932 files for validation.
Found 1870 files belonging to 155 classes.
Using 561 files for validation.


In [ ]:
train_ds = dataseta_train.concatenate(datasetb_train).concatenate(datasetc_train)
test_ds = dataseta_test.concatenate(datasetb_test).concatenate(datasetc_test)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
#train_ds = train_ds.take(330)
#test_ds = test_ds.take(330)


# Train model

In [ ]:
input_shape = (img_height, img_width, 3)
optim_1 = Adam(learning_rate=0.001)
n_epochs = 1
batch_size = 32

In [ ]:
local_train_ds = train_ds.batch(batch_size)
local_test_ds = test_ds.batch(batch_size)

### Create model

In [ ]:
def create_model(input_shape, optimizer='rmsprop', fine_tune=0):
  conv_base = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
  if fine_tune > 0:
      for layer in conv_base.layers[:-fine_tune]:
          layer.trainable = False
  else:
      for layer in conv_base.layers:
          layer.trainable = False

  top_model = conv_base.output

  top_model = Flatten(name="flatten")(top_model)
  top_model = Dense(4096, activation='relu')(top_model)
  top_model = Dense(1072, activation='relu')(top_model)
  top_model = Dropout(0.2)(top_model)
  output_layer = Dense(1, activation='sigmoid')(top_model)
  
  model = Model(inputs=conv_base.input, outputs=output_layer)

  model.compile(optimizer=optimizer, 
                loss='binary_crossentropy',
                metrics=['accuracy'])
  
  return model


### Load the received model weights and decrypt

In [ ]:
with open('model_weights_inp.pkl', 'rb') as file:
  encrypted_weights = pickle.load(file)

# decrypt HE

decrypted_weights = HE_decrypt(encrypted_weights)


### Train a local model with the decrypted weights received from the aggregation server

In [ ]:
model = create_model(input_shape, optim_1, fine_tune=0)

model.set_weights(decrypted_weights)

local_model.fit(local_train_ds, epochs=n_epochs, validation_data=local_test_ds, verbose=1)

In [ ]:
global_model.save(root_dir + "model.h5")

### Encrypt local model weights and send them to the aggregation server

In [ ]:
weights = model.get_weights()

# encrypt HE

encrypted_weights = HE_encrypt(weights)

In [ ]:
with open('model_weights_out.pkl', 'wb') as file:
  pickle.dump(encrypted_weights, file)

# Now send model_weights_out.pkl to the aggregation server